Analytic Inverse kinematic solver for simplified model of leg

In [1]:
import pybullet
import time
import pybullet_data
import numpy as np

In [23]:
phisycsClient = pybullet.connect(pybullet.GUI)
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())

Spawn the Robot...

In [24]:
pybullet.resetSimulation()
planeID = pybullet.loadURDF("plane.urdf")
pybullet.setGravity(0,0,-9.81)
robotID = pybullet.loadURDF("simple_robot/urdf/simple_robot.urdf",
                         [0.0,0.0,0.6],pybullet.getQuaternionFromEuler([0.0,0.0,0.0]),useFixedBase = 1)
pybullet.setRealTimeSimulation(1)

Check Joints + Home

In [25]:
for index in range(pybullet.getNumJoints(robotID)):
    print(pybullet.getJointInfo(robotID,index)[0:2],
          pybullet.getJointState(robotID,pybullet.getJointInfo(robotID,index)[0])[0])

homeConfig = list([0,0,-20*np.pi/180,40*np.pi/180,-20*np.pi/180,0])
for index in range(pybullet.getNumJoints(robotID)):
    pybullet.setJointMotorControl2(bodyIndex=robotID,
                            jointIndex=index,
                            controlMode=pybullet.POSITION_CONTROL,
                            targetPosition = homeConfig[index])

(0, b'base_hip_yaw_joint') -0.0003752423161250109
(1, b'hip_yaw_roll_joint') 0.00022533768659312628
(2, b'hip_roll_pitch_joint') -0.0776584757291836
(3, b'hip_knee_joint') 0.16555246445952446
(4, b'knee_ankle_pitch_joint') -0.08862138456102778
(5, b'ankle_pitch_roll_joint') -0.0003447092816232101


Inverse Kinematic Solver (Bullet Built-in)

In [15]:
## create simple trajectory for ankle
trajectory = list("")
for i in range(300):
    trajectory.append(np.array([0.0,-0.1+0.001*i,0.2]))
## Solve IK and set Joint angles
for i in range(300):
    
    config = pybullet.calculateInverseKinematics(robotID,5,
                                                 trajectory[i],pybullet.getQuaternionFromEuler([0,0,np.pi/2])) 
    """config = calculateIK(trajectory[i],[0,0,np.pi/2])"""
    for index in range(pybullet.getNumJoints(robotID)):
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index,
                                controlMode=pybullet.POSITION_CONTROL,
                                targetPosition = config[index])
    time.sleep(0.02)

Analytic IK Prerequisities

In [9]:
def transformation(pos,attitude):
    alpha,beta,gamma = attitude
    x,y,z = pos
    rot = np.array([[np.cos(alpha)*np.cos(beta),np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma),np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)],
                    [np.sin(alpha)*np.cos(beta),np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma),np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
                    [-np.sin(beta),np.cos(beta)*np.sin(gamma),np.cos(beta)*np.cos(gamma)]])
    return np.array([[rot[0][0],rot[0][1],rot[0][2],x],
                    [rot[1][0],rot[1][1],rot[1][2],y],
                    [rot[2][0],rot[2][1],rot[2][2],z],
                    [0,0,0,1]])

Analytic Solution

In [28]:
def calculateIK(pose,attitude):
    l3 = 0.165
    l4 = 0.135
    l5 = 0.0

    trans = np.linalg.inv(transformation(pose,attitude))
    # Define n',s',a',p'
    n = trans[0:3,0]
    s = trans[0:3,1]
    a = trans[0:3,2]
    p = trans[0:3,3]
    
    """print(trans)
    print(s)
    print(s)
    print(p)"""
    
    r1 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,1)[1])
    x1 = -np.cos(r1[2])*np.sin(r1[1])*np.sin(r1[0])-np.sin(r1[2])*np.cos(r1[0])
    z1 = np.cos(r1[1])*np.sin(r1[0])
    r2 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,2)[1])
    x2 = -np.cos(r2[2])*np.sin(r2[1])*np.sin(r2[0])-np.sin(r2[2])*np.cos(r2[0])
    r3 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,3)[1])
    z3 = np.cos(r3[1])*np.sin(r3[0])
    x3 = -np.cos(r3[2])*np.sin(r3[1])*np.sin(r3[0])-np.sin(r3[2])*np.cos(r3[0])
    r4 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,4)[1])
    x4 = -np.cos(r4[2])*np.sin(r4[1])*np.sin(r4[0])-np.sin(r4[2])*np.cos(r4[0])
    r5 = pybullet.getEulerFromQuaternion(pybullet.getLinkState(robotID,5)[1])
    x5 = -np.cos(r5[2])*np.sin(r5[1])*np.sin(r5[0])-np.sin(r5[2])*np.cos(r5[0])
    
    
    KNEE = np.sign(np.dot([0,0,z3],np.cross([x3,0,0],[x4,0,0])))
    ANKLE = np.sign(np.dot([0,0,z1],np.cross([x1,0,0],[x2,0,0])))
    HIP = np.sign(np.dot(p,[x5,0,0]))

    
    # Calculate Theta 4
    C4 = ((p[0] + l5)**2 + p[1]**2 + p[2]**2 - l3**2 - l4**2) / (2*l3*l4)
    S4 = KNEE * np.sqrt((1 - C4**2))
    theta4 = np.arctan2(S4,C4)
    
    # Calculate Theta 5
    theta5 = np.arctan2( -p[2],ANKLE * np.sqrt((p[0]+l5)**2 + p[1]**2)) - np.arctan2(S4 * l3, C4 * l3 + l4)
    
    # Calculate Theta 6
    theta6 = np.arctan2(p[1],-p[0] - l5)
    if (np.cos(theta4 + theta5) * l3 + np.cos(theta5) * l4) < 0:
        theta6 += np.pi
    
    # Calculate Theta 2
    C2 = np.sin(theta6) * s[0] + np.cos(theta6) * s[1]
    S2 = HIP * np.sqrt(1 - C2**2)
    theta2 = np.arctan2(S2,C2)
    
    # Calculate Theta 1
    theta1 = np.arctan2(-np.sin(theta6) * s[0] - np.cos(theta6)*s[1],
                       -np.sin(theta6) * n[0] - np.cos(theta6)*n[1])
    
    # Calculate theta 3
    theta3 = np.arctan2(a[2], np.cos(theta6) * a[0] - np.sin(theta6) * a[1]) - theta4 - theta5
    
    if S2 < 0:
        theta1 += np.pi
        theta3 += np.pi
    
    return ([theta1,theta2,theta3,theta4,theta5,theta6])

print(pybullet.calculateInverseKinematics(robotID,5,
                                        [0.0,0.1,0.45],pybullet.getQuaternionFromEuler([0,0,np.pi/2])))

print("-------------------------------------------")
print(calculateIK([0.05,0.1,0.45],[0,0,np.pi/2]))

(-0.003455616048737094, 0.7515827078433922, 1.889508099862356, -0.017026754002306542, -1.8617801660130933, -0.7533863385096443)
-------------------------------------------
[-6.80359332859641e-18, 1.5707963267948966, nan, nan, nan, -1.460139105621001]


/home/kassra/.local/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in sqrt


Analytic(analistic) IK (by me)

In [30]:
## Create Same trajectory
trajectory = list("")
for i in range(300):
    trajectory.append(np.array([0.0,-0.1+0.001*i,0.25]))
## Solve IK and set Joint angles
for i in range(300):
    
    config = calculateIK(trajectory[i],[0,0,np.pi/2])
    for index in range(pybullet.getNumJoints(robotID)):
        pybullet.setJointMotorControl2(bodyIndex=robotID,
                                jointIndex=index,
                                controlMode=pybullet.POSITION_CONTROL,
                                targetPosition = config[index])
    time.sleep(0.02)

/home/kassra/.local/lib/python3.6/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in sqrt


Shutdown

In [9]:
pybullet.resetSimulation()

In [31]:
pybullet.disconnect()

In [289]:
pybullet.setJointMotorControl2(bodyIndex=robotID,
                            jointIndex=1,
                            controlMode=pybullet.POSITION_CONTROL,
                            targetPosition = 0)